In [3]:
import torch

torch.hub.list(github="ultralytics/yolov5", trust_repo='check')

Using cache found in /home/yungshun317/.cache/torch/hub/ultralytics_yolov5_master


['custom',
 'yolov5l',
 'yolov5l6',
 'yolov5m',
 'yolov5m6',
 'yolov5n',
 'yolov5n6',
 'yolov5s',
 'yolov5s6',
 'yolov5x',
 'yolov5x6']

In [5]:
# Check model information from `hubconf.py` in `ultralytics/yolov5`
# Official model
yolo = torch.hub.load("ultralytics/yolov5", "yolov5s")
print(yolo)

Using cache found in /home/yungshun317/.cache/torch/hub/ultralytics_yolov5_master


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'setuptools>=70.0.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.5 MB/s eta 0:00:001.1 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.5 MB/s eta 0:00:00 MB/s eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 114.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.5.1
    Uninstalling setuptools-69.5.1:
      Successfully uninstalled setuptools-69.5.1

requirements: AutoUpdate success ✅ 5.4s, installed 2 packages: ['gitpython>=3.1.30', 'setuptools>=70.0.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-10-8 Python-3.12.3 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16072MiB)

100%|██████████████████████████████████████| 14.1M/14.1M [00:00<00:00, 17.5MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [13]:
from torchinfo import summary

# Peek inside the nested model
summary(yolo.model.model, input_size=(1, 3, 640, 640))

Layer (type:depth-idx)                        Output Shape              Param #
DetectionModel                                [1, 25200, 85]            --
├─Sequential: 1-1                             --                        --
│    └─Conv: 2-1                              [1, 32, 320, 320]         --
│    │    └─Conv2d: 3-1                       [1, 32, 320, 320]         (3,488)
│    │    └─SiLU: 3-2                         [1, 32, 320, 320]         --
│    └─Conv: 2-2                              [1, 64, 160, 160]         --
│    │    └─Conv2d: 3-3                       [1, 64, 160, 160]         (18,496)
│    │    └─SiLU: 3-4                         [1, 64, 160, 160]         --
│    └─C3: 2-3                                [1, 64, 160, 160]         --
│    │    └─Conv: 3-5                         [1, 32, 160, 160]         (2,080)
│    │    └─Sequential: 3-6                   [1, 32, 160, 160]         (10,304)
│    │    └─Conv: 3-7                         [1, 32, 160, 160]         (

In [ ]:
# Use `export.py` for converting `yolov5s.pt` to ONNX from Ultralytics's YOLOv5 repository

# !cd yolov5
# !git clone https://github.com/ultralytics/yolov5
# !cd yolov5
# !pip3 install ultralytics
# !python export.py --weights yolov5s.pt --workspace 2048 --data data/coco.yaml --include onnx --opset 12

# This will output a `yolov5s.onnx` file

In [ ]:
# /usr/src/tensorrt/bin/trtexec --onnx=yolov5s.onnx --saveEngine=/tensorfl_vision/Onnx-Inference-Yolov5/models_engine/yolov5s.engine